# Quasistatic Planar Creating Desired Trajectories

## Imports

In [1]:
import time
import os
import numpy as np

from pydrake.all import (PiecewisePolynomial, ModelInstanceIndex,
    RotationMatrix, RigidTransform, Rgba, Box, Sphere, BaseField,
    Evaluate, Fields, PointCloud, MeshcatAnimation)

from qsim.parser import (
    QuasistaticParser,
    QuasistaticSystemBackend,
    GradientMode,
)

from qsim.simulator import ForwardDynamicsMode, InternalVisualizationType
from qsim.model_paths import models_dir, package_paths_dict

/home/shaoyuan/Documents/Software/quasistatic_simulator/robotics_utilities/qp_derivatives/qp_derivatives.py:2: DrakeDeprecationWarning: Please import from the pydrake.solvers module directly, instead of the deprecated pydrake.solvers.mathematicalprogram submodule. The deprecated code will be removed from Drake on or after 2023-05-01.
  import pydrake.solvers.mathematicalprogram as mp


## Initialize Quasistatic Simulator

In [2]:
# q_model_path = os.path.join(models_dir, "q_sys", "box_pushing.yml")
package_paths_dict["dccm_quasistatic"] =  "/home/shaoyuan/Documents/Software/dccm_quasistatic"
q_model_path = os.path.join("../../models", "q_sys", "box_pushing.yml")
q_parser = QuasistaticParser(q_model_path)
q_sim = q_parser.make_simulator_cpp()
q_sim_py = q_parser.make_simulator_py(InternalVisualizationType.Cpp)
q_sim_batch = q_parser.make_batch_simulator()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
meshcat = q_sim_py.meshcat
def draw_pose_of_box(name, pos, color):
    pose = RigidTransform(RotationMatrix().MakeXRotation(pos[-1]), np.concatenate(([0], pos[0:2])))
    
    meshcat.SetObject(name, Box(1,1,1), rgba=color)
    meshcat.SetTransform(name, pose)


def visualize_open_loop_traj(q0, q_desired, sim_params):
    q = q0
    actual = [q0]
    for i, q_des in enumerate(q_desired):
        if i == len(q_desired) - 1:
            break
        u_desired = q_desired[i+1, -2:]
        b_des = q_des[:3]
        draw_pose_of_box("b_des", b_des, Rgba(1, 0, 0, 1))
        q_sim_py.update_mbp_positions_from_vector(q)
        q_sim_py.draw_current_configuration()
        time.sleep(0.05)
        q = q_sim.calc_dynamics(q, u_desired, sim_params)
        actual.append(q)
    return np.array(actual)

def make_desired_traj_achievable(q_desired, sim_params):
    actual = [q_desired[0]]
    q = q_desired[0]
    for i, q_des in enumerate(q_desired):
        if i == len(q_desired) - 1:
            u_desired = q_desired[i, -2:]
        else:
            u_desired = q_desired[i+1, -2:]
        b_des = q_des[:3]
        q_sim_py.update_mbp_positions_from_vector(q)
        q_sim_py.draw_current_configuration()
        time.sleep(0.05)
        q = q_sim.calc_dynamics(q, u_desired, sim_params)
        actual.append(q)
    return np.array(actual)

def make_idealized_traj(b_r, s_r, theta_buff, n_steps):
    q_desired = []
    for i in range(n_steps):
        b_theta = np.pi * 2 * i / n_steps
        b_x = b_r * np.sin(b_theta)
        b_y = b_r * np.cos(b_theta)
        s_theta = b_theta - theta_buff
        s_x = s_r * np.sin(s_theta)
        s_y = s_r * np.cos(s_theta)
        q = np.array([b_x, b_y, -b_theta, s_x, s_y])
        q_desired.append(q)
        q_sim_py.update_mbp_positions_from_vector(q)
        # q_sim_py.draw_current_configuration()
        # time.sleep(0.03)
        # q_next = q_sim.calc_dynamics(q, q[-2:], sim_params)
        
        # q_sim_py.update_mbp_positions_from_vector(q_next)
        # q_sim_py.draw_current_configuration()
        # time.sleep(0.3)
    return np.array(q_desired)

## Creating q_desired for planar pushing (Log barrier weight 10)

In [5]:
# This trajectory is s_r is not large enough

sim_params = q_sim.get_sim_params_copy()
sim_params.h = 0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 10

q_desired = []
n_steps = 100
b_r = 1.5
s_r = b_r + 1.05
theta_buff = 0.9
for i in range(n_steps):
    b_theta = np.pi * 2 * i / n_steps
    b_x = b_r * np.sin(b_theta)
    b_y = b_r * np.cos(b_theta)
    s_theta = b_theta - theta_buff
    s_x = s_r * np.sin(s_theta)
    s_y = s_r * np.cos(s_theta)
    q = np.array([b_x, b_y, -b_theta, s_x, s_y])
    q_desired.append(q)
    q_sim_py.update_mbp_positions_from_vector(q)
    q_sim_py.draw_current_configuration()
    time.sleep(0.05)
    # q_next = q_sim.calc_dynamics(q, q[-2:], sim_params)
    
    # q_sim_py.update_mbp_positions_from_vector(q_next)
    # q_sim_py.draw_current_configuration()
    # time.sleep(0.3)
q_desired = np.array(q_desired)


In [ ]:
# This trajectory of u_desired works open loop
# Most ideal trajectory for b_r = 1.5

sim_params = q_sim.get_sim_params_copy()
sim_params.h =0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 10

q_desired = []
n_steps = 100
b_r = 1.5
s_r = b_r + 1.1
theta_buff = 0.9
for i in range(n_steps):
    b_theta = np.pi * 2 * i / n_steps
    b_x = b_r * np.sin(b_theta)
    b_y = b_r * np.cos(b_theta)
    s_theta = b_theta - theta_buff
    s_x = s_r * np.sin(s_theta)
    s_y = s_r * np.cos(s_theta)
    q = np.array([b_x, b_y, -b_theta, s_x, s_y])
    q_desired.append(q)
    q_sim_py.update_mbp_positions_from_vector(q)
    q_sim_py.draw_current_configuration()
    time.sleep(0.01)
    # q_next = q_sim.calc_dynamics(q, q[-2:], sim_params)
    
    # q_sim_py.update_mbp_positions_from_vector(q_next)
    # q_sim_py.draw_current_configuration()
    # time.sleep(0.3)
q_desired = np.array(q_desired)


In [ ]:
# s_r is too large

sim_params = q_sim.get_sim_params_copy()
sim_params.h =0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 10

q_desired = []
n_steps = 100
b_r = 1.5
s_r = b_r + 1.6
theta_buff = 0.9
for i in range(n_steps):
    b_theta = np.pi * 2 * i / n_steps
    b_x = b_r * np.sin(b_theta)
    b_y = b_r * np.cos(b_theta)
    s_theta = b_theta - theta_buff
    s_x = s_r * np.sin(s_theta)
    s_y = s_r * np.cos(s_theta)
    q = np.array([b_x, b_y, -b_theta, s_x, s_y])
    q_desired.append(q)
    q_sim_py.update_mbp_positions_from_vector(q)
    q_sim_py.draw_current_configuration()
    time.sleep(0.01)
    # q_next = q_sim.calc_dynamics(q, q[-2:], sim_params)
    
    # q_sim_py.update_mbp_positions_from_vector(q_next)
    # q_sim_py.draw_current_configuration()
    # time.sleep(0.3)
q_desired = np.array(q_desired)


In [ ]:
# This trajectory of u_desired works open loop
# Most ideal trajectory for b_r = 1.2

q_desired = []
n_steps = 100
b_r = 1.2
s_r = b_r + 1.7
theta_buff = 0.7
for i in range(n_steps):
    b_theta = np.pi * 2 * i / n_steps
    b_x = b_r * np.sin(b_theta)
    b_y = b_r * np.cos(b_theta)
    s_theta = b_theta - theta_buff
    s_x = s_r * np.sin(s_theta)
    s_y = s_r * np.cos(s_theta)
    q = np.array([b_x, b_y, -b_theta, s_x, s_y])
    q_desired.append(q)
    q_sim_py.update_mbp_positions_from_vector(q)
    q_sim_py.draw_current_configuration()
    time.sleep(0.01)
    # q_next = q_sim.calc_dynamics(q, q[-2:], sim_params)
    
    # q_sim_py.update_mbp_positions_from_vector(q_next)
    # q_sim_py.draw_current_configuration()
    # time.sleep(0.3)
q_desired = np.array(q_desired)

Running that desired trajectory open loop

In [ ]:
q = q_desired[0]
visualize_open_loop_traj(q, q_desired, sim_params)

Visualizing open loop control on the spliced trajectories including initial disturbance


In [ ]:


n_steps = 100
b_r = 1.4
s_r = b_r + 1.3
theta_buff = 0.9
q_desired_1 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_1 = make_desired_traj_achievable(np.array(q_desired_1), sim_params=sim_params)

# visualize_open_loop_traj(q_desired_1[0], q_desired_1)

b_r = 1.5
s_r = b_r + 1.1
theta_buff = 0.9
q_desired_2 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_2 = make_desired_traj_achievable(np.array(q_desired_2), sim_params=sim_params)
# visualize_open_loop_traj(q_desired_2[0], q_desired_2)

# Splice the the two trajectories together
break_idx = int(n_steps/3)
# lower, higher, lower
# q_desired = np.concatenate((q_desired_1[:break_idx], q_desired_2[break_idx:break_idx*2], q_desired_1[break_idx*2:]), axis=0)
# higher, lower, higher
# q_desired = np.concatenate((q_desired_2[:break_idx], q_desired_1[break_idx:break_idx*2], q_desired_2[break_idx*2:]), axis=0)

break_idx = int(n_steps/2)
# lower, higher # this one seems to correct itself not a good example
# q_desired = np.concatenate((q_desired_1[:break_idx], q_desired_2[break_idx:]), axis=0)
# higher, lower # this is a pretty good test case, see if the controller can achieve lower error than this open loop traj
traj_id = "A"
q_desired = np.concatenate((q_desired_2[:break_idx], q_desired_1[break_idx:]), axis=0)
# q_desired = q_desired_2

initial_offset = np.array([0.2, -0.1, 0.1, 0.1, 0.2])
q = q_desired[0] + initial_offset
visualize_open_loop_traj(q, q_desired)

In [ ]:
# adding the initial state to the traj data
import zarr
import numpy as np
traj_id = "A"
traj_file_name = f"desired_trajectories/{traj_id}.zarr"
traj = zarr.open_group(traj_file_name, mode='a')
q_desired = traj["q_desired"]
initial_offset = np.array([0.2, -0.1, 0.1, 0.1, 0.2])
q0 = q_desired[0] + initial_offset
traj["q0"] = q0

In [7]:
# adding open loop control to the traj data
import zarr
import numpy as np
traj_id = "A"
traj_file_name = f"desired_trajectories/{traj_id}.zarr"
traj = zarr.open_group(traj_file_name, mode='a')
traj["q_openloop"] = visualize_open_loop_traj(traj["q0"][:], traj["q_desired"][:], sim_params) 


## Log barrier weight 100

### B

In [ ]:
# def make_desired_traj_achievable(q_desired, sim_params):
#     actual = [q_desired[0]]
#     q = q_desired[0]
#     for i, q_des in enumerate(q_desired):
#         if i == len(q_desired) - 1:
#             u_desired = q_desired[i, -2:]
#         else:
#             u_desired = q_desired[i+1, -2:]
#         b_des = q_des[:3]
#         q_sim_py.update_mbp_positions_from_vector(q)
#         q_sim_py.draw_current_configuration()
#         time.sleep(0.05)
#         q = q_sim.calc_dynamics(q, u_desired, sim_params)
#         actual.append(q)
#     return np.array(actual)

def make_desired_traj_achievable(q_desired, sim_params):
    actual = q_desired
    q = q_desired[0]
    for i, q_des in enumerate(q_desired):
        if i == len(q_desired) - 1:
            u_desired = q_desired[i, -2:]
        else:
            u_desired = q_desired[i+1, -2:]
        b_des = q_des[:3]
        actual[i, :3] = q[:3]
        q_sim_py.update_mbp_positions_from_vector(q)
        q_sim_py.draw_current_configuration()
        # time.sleep(0.05)
        q = q_sim.calc_dynamics(q, u_desired, sim_params)
        
    return actual

def make_idealized_traj(b_r, s_r, theta_buff, n_steps):
    q_desired = []
    for i in range(n_steps):
        b_theta = np.pi * 2 * i / n_steps
        b_x = b_r * np.sin(b_theta)
        b_y = b_r * np.cos(b_theta)
        s_theta = b_theta - theta_buff
        s_x = s_r * np.sin(s_theta)
        s_y = s_r * np.cos(s_theta)
        q = np.array([b_x, b_y, -b_theta, s_x, s_y])
        q_desired.append(q)
        q_sim_py.update_mbp_positions_from_vector(q)
    return np.array(q_desired)

In [ ]:
sim_params = q_sim.get_sim_params_copy()
sim_params.h =0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 100

n_steps = 100
b_r = 1.2
s_r = b_r + 0.21298
theta_buff = 0.45
q_desired_1 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_1 = make_desired_traj_achievable(q_desired_1, sim_params=sim_params)

visualize_open_loop_traj(q_desired_1[0], q_desired_1, sim_params)

In [ ]:
n_steps = 100
b_r = 1.0
s_r = b_r + 0.2445
theta_buff = 0.5
q_desired_2 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_2 = make_desired_traj_achievable(q_desired_2, sim_params=sim_params)

visualize_open_loop_traj(q_desired_2[0], q_desired_2, sim_params)

In [ ]:
break_idx = int(n_steps/2)
# lower, higher # this one seems to correct itself not a good example
# q_desired = np.concatenate((q_desired_1[:break_idx], q_desired_2[break_idx:]), axis=0)
# higher, lower # this is a pretty good test case, see if the controller can achieve lower error than this open loop traj
traj_id = "B"
q_desired = np.concatenate((q_desired_2[:break_idx], q_desired_1[break_idx:]), axis=0)
# q_desired = q_desired_2

initial_offset = np.array([0.1, -0.1, 0.05, 0.1, 0.05])
q = q_desired[0] + initial_offset
# q = q_desired[0]
visualize_open_loop_traj(q, q_desired, sim_params=sim_params)

In [ ]:
import zarr
import numpy as np
traj_id = "B"
traj_file_name = f"desired_trajectories/{traj_id}.zarr"
traj = zarr.open_group(traj_file_name, mode='a')
q_desired = traj["q_desired"]
# initial_offset = np.array([0.2, -0.1, 0.1, 0.1, 0.2])
# q0 = q_desired[0] + initial_offset
traj["q0"] = q_desired[0]

### C

In [ ]:
sim_params = q_sim.get_sim_params_copy()
sim_params.h =0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 100

n_steps = 100
b_r = 1.2
s_r = b_r + 0.21298
theta_buff = 0.45
q_desired_1 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_1 = make_desired_traj_achievable(q_desired_1, sim_params=sim_params)

visualize_open_loop_traj(q_desired_1[0], q_desired_1, sim_params)

In [ ]:
n_steps = 100
b_r = 1.1
s_r = b_r + 0.224
theta_buff = 0.45
q_desired_2 = make_idealized_traj(b_r, s_r, theta_buff, n_steps)
q_desired_2 = make_desired_traj_achievable(q_desired_2, sim_params=sim_params)

visualize_open_loop_traj(q_desired_2[0], q_desired_2, sim_params)

In [ ]:
break_idx = int(n_steps/2)
# lower, higher 
# q_desired = np.concatenate((q_desired_1[:break_idx], q_desired_2[break_idx:]), axis=0)
# higher, lower
traj_id = "C"
q_desired = np.concatenate((q_desired_2[:break_idx], q_desired_1[break_idx:]), axis=0)
# q_desired = q_desired_2

initial_offset = np.array([0.01, -0.01, 0.01, 0.01, -0.01])
q = q_desired[0] + initial_offset
# q = q_desired[0]
visualize_open_loop_traj(q, q_desired, sim_params=sim_params)

In [ ]:
import zarr
traj_file_name = f"desired_trajectories/{traj_id}.zarr"
traj = zarr.open_group(traj_file_name, mode='w')
traj["q_desired"] = q_desired
traj["q0"] = q

In [ ]:
traj = zarr.open_group(traj_file_name, mode='r')
q_desired = traj["q_desired"]
q0 = traj["q0"]
visualize_open_loop_traj(q0, q_desired, sim_params=sim_params)